# Part 2: Decision Trees

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score

First, load in the data, and drop all rows that have NaN values. Then create X and y
numpy arrays, where X is a matrix so that all of the rows are the different rows from the
dataframes and the columns are the features listed above, and y is the numpy array of the
Status column. Once you have completed this, X and y should both have 1649 rows.
The next step is to create a train test split on the data. We’re going to use a test size of
0.33 and a random state of 42.

In [7]:
# Read in CSV here
life_expectancy_data = pd.read_csv("Life Expectancy Data.csv")
life_expectancy_data = life_expectancy_data.dropna()

X = np.c_[life_expectancy_data["Hepatitis_B"], 
            life_expectancy_data["Polio"],
            life_expectancy_data["Measles"],
            life_expectancy_data["BMI"],
            life_expectancy_data["Diphtheria"],
            life_expectancy_data["HIV/AIDS"],
            life_expectancy_data["Adult_Mortality"]]

y = np.array(life_expectancy_data["Status"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


Question 2.1 (10 points): What is the information gain if the split is on BMI < 50? Look
at slides from lecture on calculating information gain to use the correct formula

Question 2.2 (10 points): Create a decision tree classifier with the entropy criterion and fit
to the training data. What is the accuracy score of this classifier on the testing set? What
is the first decision that is used to split the data (which feature does it use and what value
does it split on)?

Question 2.3 (10 points): Create decision trees with max depths from 2 to 20 and calculate
the training and testing accuracy for each decision tree and display in a table. Which one is
best? Provide a short (1-2 sentence) explanation for why that depth is the best performing.

Question 2.4 (5 points): Show the diagram of the decision three with max depth of 2.

Question 2.5 (5 points): For the best performing tree in Question 2.3 (in terms of testing
results), show the confusion matrices that are produced for the training set and testing set.

Question 2.6 (15 points): Create two additional decision trees, where min samples leaf set
is set to 10 and min samples split set to 2 and a second where min samples leaf set is set
to 10 and min samples split set to 30. Calculate the training and testing scores for each.
Which of these performs better? Provide a short explanation (2-3 sentences) for why one
performs better than the other.